In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import time

# Set up the WebDriver (choose your preferred browser)
service = Service(executable_path="chromedriver.exe")
options = webdriver.ChromeOptions()



# Add other desired options here

driver = webdriver.Chrome(options=options)
driver.get('https://www.bls.gov/ooh/home.htm')
driver.maximize_window()
time.sleep(5)



In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd
import time

# Set up the WebDriver (choose your preferred browser)
service = Service(executable_path="chromedriver.exe")
options = webdriver.ChromeOptions()
# Add other desired options here

# Start the WebDriver
driver = webdriver.Chrome(service=service, options=options)

# Open the webpage
driver.get('https://www.bls.gov/ooh/home.htm')

# Wait for some time to ensure the page is fully loaded
time.sleep(5)

# Get the page source
page_source = driver.page_source

# Parse the page source using BeautifulSoup
soup = BeautifulSoup(page_source, 'html.parser')

# Find all <a> tags with href containing '/ooh/'
links = soup.find_all('a', href=lambda href: href and '/ooh/' in href)

# Extract the href and text of each link
link_data = []
for link in links:
    link_data.append({'href': link['href'], 'text': link.text.strip()})

# Convert the list of dictionaries to a pandas DataFrame
df = pd.DataFrame(link_data)

# Close the WebDriver
driver.quit()

# Display the DataFrame
print(df)



In [ ]:
# Save the DataFrame to a CSV file
df.to_csv('links.csv', index=False)

In [ ]:
import pandas as pd
pd_links = pd.read_csv('links.csv')
pd_links = pd_links[~pd_links['text'].str.contains('view', na=False)]
# Drop rows with index from 0 to 9
pd_links.drop(index=pd_links.index[:9], inplace=True)

# Drop rows with index from 700 to 750
pd_links.drop(index=pd_links.index[358:], inplace=True)
# Add a new column named 'type' and set initial value as 'occupation'
pd_links['type'] = 'occupation'

# Mark the first 23 rows as 'group' in the 'type' column
pd_links.loc[:33, 'type'] = 'group'
pd_links['grp'] = pd_links['href'].str.split('/').str[2]

vl = pd_links.iloc[:25, [1, 3]].copy()

pd_links = pd_links.merge(vl, on='grp', how='left')

pd_links.rename(columns={'text_y': 'group'}, inplace=True)
pd_links.rename(columns={'text_x': 'Occupation'}, inplace=True)

pd_links_sorted = pd_links.sort_values(by=['group', 'type', 'Occupation'])

In [ ]:
# Save the DataFrame to a CSV file
pd_links.to_csv('BLSdata.csv', index=False)

In [ ]:
import pandas as pd
BLSdata = pd.read_csv('BLSdata.csv')
OCCdata =  BLSdata[BLSdata['type']=='occupation']

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

for url in OCCdata['href']:
    page = f'https://www.bls.gov{url}'
    response = requests.get(page)
    soup = BeautifulSoup(response.content, 'html.parser')

    


In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Create empty lists to store extracted data
median_pay = []
hourly_pay = []

for url in OCCdata['href']:
    page = f'https://www.bls.gov{url}'
    response = requests.get(page)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # Find all <td> elements
    td_elements = soup.find_all('td')
    found = False
    
    # Iterate through all <td> elements to find median pay
    for td_element in td_elements:
        if "Median Pay" in td_element.text:
            found = True
            data = td_element.text.strip()
            median_pay.append(data)

            # Extract hourly pay if available
            br_element = td_element.find_next('br')
            if br_element:
                hourly_pay_text = br_element.next_sibling.strip()
                hourly_pay.append(hourly_pay_text)
            else:
                hourly_pay.append(None)
            break
    
    # If median pay is not found, append None for both median_pay and hourly_pay
    if not found:
        median_pay.append(None)
        hourly_pay.append(None)

# Create a DataFrame from the extracted data
data = {
    'Median Pay': median_pay,
    'Hourly Pay': hourly_pay
}

df = pd.DataFrame(data)

# Now you have a DataFrame containing the extracted data
print(df)


In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Create an empty DataFrame to store extracted data (modify column names as needed)
job_data_df = pd.DataFrame(columns=['URL', 'Median Pay', 'Education', 'Experience'])

for url in OCCdata['href']:
  page = f'https://www.bls.gov{url}'
  response = requests.get(page)
  soup = BeautifulSoup(response.content, 'html.parser')

  # Extract data
  job_data = {}
  job_data['URL'] = url  # Store URL for reference

  # Target table row for Median Pay
  table_row = soup.find('tr', {'class': 'quickfacts'})  # Look for rows with class 'quickfacts'
  if table_row:
    data_cell = table_row.find('th', text=lambda text: text and text.startswith('2022 Median Pay'))  # Find header containing '2022 Median Pay'
    if data_cell:
      # Get the text content within the table cell, excluding child elements (anchor tag)
      job_data['Median Pay'] = data_cell.text.strip().split('\n')[0]  # Get first line, remove whitespace
    else:
      job_data['Median Pay'] = ""  # Assign empty string if header not found

  # Target table row for Education (similar approach)
  table_row = soup.find('tr', {'class': 'quickfacts'})
  if table_row:
    data_cell = table_row.find('th', text=lambda text: text and text.startswith('Typical Entry-Level Education'))
    if data_cell:
      job_data['Education'] = data_cell.text.strip().split('\n')[0]  # Get first line, remove whitespace
    else:
      job_data['Education'] = ""

  # Target table row for Experience (similar approach)
  table_row = soup.find('tr', {'class': 'quickfacts'})
  if table_row:
    data_cell = table_row.find('th', text=lambda text: text and text.startswith('Work Experience'))
    if data_cell:
      job_data['Experience'] = data_cell.text.strip().split('\n')[0]  # Get first line, remove whitespace
    else:
      job_data['Experience'] = ""

  # ... Extract and store other desired data points ...

  # Append data to DataFrame
  job_data_df = pd.concat([job_data_df, pd.DataFrame([job_data])], ignore_index=True)

print(job_data_df)  # Print the DataFrame containing extracted data (may include NaN for missing data)


                                                   URL Median Pay Education  \
0    /ooh/personal-care-and-service/funeral-service...        NaN       NaN   
1    /ooh/business-and-financial/insurance-underwri...        NaN       NaN   
2    /ooh/computer-and-information-technology/compu...        NaN       NaN   
3       /ooh/math/mathematicians-and-statisticians.htm        NaN       NaN   
4    /ooh/life-physical-and-social-science/atmosphe...        NaN       NaN   
..                                                 ...        ...       ...   
328  /ooh/transportation-and-material-moving/taxi-d...        NaN       NaN   
329  /ooh/community-and-social-service/community-he...        NaN       NaN   
330           /ooh/healthcare/medical-dosimetrists.htm        NaN       NaN   
331  /ooh/installation-maintenance-and-repair/calib...        NaN       NaN   
332  /ooh/management/entertainment-and-recreation-m...        NaN       NaN   

    Experience  
0          NaN  
1          NaN  
